<a href="https://colab.research.google.com/github/danielbauer1860/LDS_Project/blob/main/fine_tuning/Llama_2_fine_tuning_aca_4epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import pandas as pd
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    )
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
import transformers

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Linguistic Data Science/data/bnc_baby_texts.csv', sep='|')
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Linguistic Data Science/data/bnc_baby_texts.csv'

In [ ]:
df_aca, df_dem, df_fic, df_news = [y for x, y in df.groupby(['category'])]

In [ ]:
df_aca

In [ ]:
aca_dataset = Dataset.from_pandas(df_aca)
fic_dataset = Dataset.from_pandas(df_fic)
news_dataset = Dataset.from_pandas(df_news)

In [ ]:
aca_dataset

# Fine-tuning Llama 2

Norouzi (2023) describes an efficient way to handle Llama 2 despite the system RAM limitations that Google Colab comes with and was, therefore, used as the main source throughout this section. For more details see: [Mastering Llama 2: A Comprehensive Guide to Fine-Tuning in Google Colab](https://artificialcorner.com/mastering-llama-2-a-comprehensive-guide-to-fine-tuning-in-google-colab-bedfcc692b7f). Additionally, [this notebook](https://colab.research.google.com/drive/12dVqXZMIVxGI0uutU6HG9RWbWPXL3vts?authuser=2#scrollTo=qmA4G6C64dJ4) was used as a reference for further parameters.

In [ ]:
# Declaring the model name; as described in the paper, the 7 billion parameter version of Llama 2 is used
model_name = "meta-llama/Llama-2-7b-hf"

The parameters for the quantization config need to be defined:

In [ ]:
# Load the entire model on the GPU 0
device_map = {"": 0}

# Set base model loading in 4-bits
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = torch.float16

# Quantization type (fp4 or nf4); nf4 is shown to be better in the QLoRA paper
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [ ]:
# Initialize the quantization config using the previously declared parameters
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)

Now that we got both the `device_map` set to one GPU and the quantization config `bnb_config` initialized, the base model can be loaded into the system without any memory issues:

In [ ]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=bnb_config,
)
model.config.use_cache = False

In [ ]:
# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

As the original tokenizer has `pad_id` set to -1, we need to to define a custom padding token. While the [Llama 2 huggingface documentation](https://huggingface.co/docs/transformers/main/model_doc/llama2#overview) recommends using `tokenizer.add_special_tokens({"pad_token":"<pad>"})`, Norouzi points out that this can introduce CUDA-related errors. Therefore, he sees directly setting the `pad_token` as a safer option.




In [ ]:
#Define a custom padding token
tokenizer.pad_token = tokenizer.eos_token

# Set the padding direction to the right
tokenizer.padding_side = "right"

These parameters are set in accordance to the QLoRA method.

In [ ]:
# LoRA attention dimension
lora_r = 64
# Alpha for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA
lora_dropout = 0.1

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
output_dir = "/content/drive/MyDrive/results/academic"
final_checkpoint_dir = os.path.join(output_dir, "2e_final_checkpoint")

While most hyperparameters remain the same throughout the different variants, the number of epochs will be varied from 2 to 4 and 8.

In [ ]:
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
num_train_epochs = 4 #2, 8
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 5
learning_rate = 4e-5
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "constant"

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    num_train_epochs=num_train_epochs
)

In [ ]:
max_seq_length = 2500
packing = False

In [ ]:
aca_trainer = SFTTrainer(
    model=model,
    train_dataset=aca_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments
)

In [ ]:
resume_checkpoint = False

In [ ]:
transformers.logging.set_verbosity_info()

In [ ]:
for name, module in aca_trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
aca_trainer.train(resume_checkpoint)

Saving and uploading the model:

In [ ]:
model_to_save = aca_trainer.model.module if hasattr(aca_trainer.model, 'module') else aca_trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
model.push_to_hub(f"dbauer1860/llama-2-bnc-baby-academic-{num_train_epochs}-epochs", create_pr=1)

Exporting the relevant log information to plot in another notebook

In [ ]:
aca_trainer.state.log_history

In [ ]:
loss_values = []
epochs = []

for step in aca_trainer.state.log_history:
  for k, v in step.items():
    if k == 'loss' and v not in loss_values:
      loss_values.append(v)
    if k == 'epoch'and v not in epochs:
      epochs.append(v)

print(epochs)

In [ ]:
df = pd.DataFrame({'loss': loss_values, 'epoch': epochs})
df

In [ ]:
df.to_csv("/content/drive/MyDrive/Linguistic Data Science/data/logs/academic_4e_logs.csv", sep=";", index=False)